<a href="https://colab.research.google.com/github/Arun-Alpy/PySpark/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Welcome")

Welcome
